In [28]:
# Import necessary libraries
import numpy as np
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Add, ReLU, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.optimizers import Adam

In [29]:
# Load the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()


In [30]:
# Reshape the data to have 1 channel (grayscale)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [31]:
# Normalize the data to [0, 1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [32]:
# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [33]:
# Verify the shapes of the data
print("Training data shape:", X_train.shape)  # Should be (60000, 28, 28, 1)
print("Training labels shape:", y_train.shape)  # Should be (60000, 10)
print("Test data shape:", X_test.shape)  # Should be (10000, 28, 28, 1)
print("Test labels shape:", y_test.shape)  # Should be (10000, 10)

Training data shape: (60000, 28, 28, 1)
Training labels shape: (60000, 10)
Test data shape: (10000, 28, 28, 1)
Test labels shape: (10000, 10)


In [36]:
# Function to build a residual block
def residual_block(x, filters, kernel_size=3, stride=1):
    # Shortcut connection
    shortcut = x
    
    # Main path
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    # If shape mismatch, use a convolution to match the dimensions
    if stride != 1:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    # Add the shortcut (residual connection)
    x = Add()([x, shortcut])
    x = ReLU()(x)
    
    return x

In [37]:
# Build a ResNet-inspired model
def build_resnet(input_shape):
    inputs = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv2D(64, (3, 3), strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Add residual blocks
    x = residual_block(x, 64, stride=1)
    x = residual_block(x, 64, stride=2)
    
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128, stride=1)
    
    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)
    
    # Fully connected layer for classification
    outputs = Dense(10, activation='softmax')(x)
    
    # Define the model
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [38]:
# Instantiate the ResNet model
model = build_resnet((28, 28, 1))

In [39]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [40]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)


Epoch 1/10
1500/1500 [==============================] - 265s 175ms/step - loss: 0.4383 - accuracy: 0.8408 - val_loss: 0.3384 - val_accuracy: 0.8773
Epoch 2/10
1500/1500 [==============================] - 264s 176ms/step - loss: 0.2807 - accuracy: 0.8970 - val_loss: 0.2598 - val_accuracy: 0.9097
Epoch 3/10
1500/1500 [==============================] - 263s 176ms/step - loss: 0.2327 - accuracy: 0.9160 - val_loss: 0.2505 - val_accuracy: 0.9107
Epoch 4/10
1500/1500 [==============================] - 289s 193ms/step - loss: 0.2029 - accuracy: 0.9258 - val_loss: 0.2734 - val_accuracy: 0.9044
Epoch 5/10
1500/1500 [==============================] - 285s 190ms/step - loss: 0.1758 - accuracy: 0.9359 - val_loss: 0.3209 - val_accuracy: 0.8856
Epoch 6/10
1500/1500 [==============================] - 291s 194ms/step - loss: 0.1534 - accuracy: 0.9444 - val_loss: 0.3170 - val_accuracy: 0.8951
Epoch 7/10
1500/1500 [==============================] - 351s 234ms/step - loss: 0.1310 - accuracy: 0.9520 - val_

In [41]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

313/313 [==============================] - 14s 44ms/step - loss: 0.2700 - accuracy: 0.9246
Test loss: 0.2699919044971466, Test accuracy: 0.9246000051498413


In [43]:
# Save the model
model.save('fashion_mnist_resnet_model.keras')

In [44]:
# Save the training history
np.save('resnet_model_history.npy', history.history)

In [48]:
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)

In [49]:
# Save the model in the specified folder
model.save(os.path.join(model_dir, 'fashion_mnist_resnet_model.keras'))

In [50]:
# Save the training history in the specified folder
np.save(os.path.join(model_dir, 'resnet_model_history.npy'), history.history)